<!-- dom:TITLE: Covid-19, data and a simple compartment model -->
# Covid-19, data and a simple compartment model
<!-- dom:AUTHOR: Aksel Hiorth and Oddbjørn Nødland -->
<!-- Author: -->  
**Aksel Hiorth and Oddbjørn Nødland**

Date: **Aug 19, 2021**

In [1]:
%matplotlib inline

# Import the packages we plan to use
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**Abstract.** In this project, a simple $SI$ (Susceptible, Infected)-model is used to analyze
the first 250 days of the Corona virus outbreak. In areas with high population
density, a model with a constant disease transmission rate, $\beta$, fits the
data well, provided we match the total number of infected people to the data.
In areas with a lower population density, as in the Nordic countries,
the same model works poorly.
Extending the model to allow for a time-dependent $\beta$ makes it possible
to account for varying governmental responses among countries.
The results presented here supports the notion that the governmental
response was stronger in Hubei, China compared to Norway.




# Introduction

The Corona virus disease, COVID-19, is brought on by infectious contact with
*SARS-CoV-2, severe acute respiratory syndrome coronavirus 2* [[SarsCov2]](#SarsCov2).
Since its original discovery in Wuhan, China [[zhou2020pneumonia]](#zhou2020pneumonia), it
spread rapidly to most of the world's countries and has cause a pandemic
we are still in the middle of.

In this project we investigate the 250 first days of the Corona virus outbreak.
The project is divided into two parts. In the first part, we import and view
data on reported number of confirmed cases using the Python libraries `pandas`
and `matplotlib`. The data can be freely downloaded from the Johns Hopkins
git repository,
[Center for Systems Science and Engineering (CSSE) at
Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19).
In the second part, we use a mathematical compartment model to interpret
the data.
Finally, we present our conclusions as well as reflections about what we
learned working on the project.



# Exercise 1



**Part 1.**

We start by reading the contents of the text file `corona_data.dat` into a
`pandas.DataFrame` object:

In [2]:
df = pd.read_csv('../data/corona_data.dat', sep='\t')

In the text file, columns of data are separated by a tab (`\t`) marker.
Therefore, we have to use the command `sep='\t'` in order for the `read_csv`
function to work properly
(as explained in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html))


**Part 2.**

The data frame, `df`, now contains COVID-19 data from all countries,
as well as several regions and other kinds of locations. If we want
to extract only the data for a specific country, say Afghanistan,
we can also do that very easily with `pandas`:

In [3]:
df = df[df['LOCATION'] == 'Afghanistan']

The innermost command (`df['LOCATION'] == 'Afghanistan'`) returns an array
of boolean variables (i.e., either `True` or `False`) signifying which entries
have "Afghanistan" in the location column. Since `pandas` supports
[boolean masking](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html),
this array can in turn be passed to the original data frame,
giving back a sub-dataframe containing only the rows corresponding to
`True`.


**Part 3.**

Since we are interested in comparing data for different locations, we create
a Python function that returns the data for one specific location:

In [4]:
def get_corona_data(location, data_file='../data/corona_data.dat'):
    """
    Extracts COVID-19 data for a specific location.

    :param location: The name of the location (case-sensitive).
    :param data_file: Path to file holding the COVID-19 data.
                      It is expected that columns of data are
                      separated by tabs, and that there is a
                      column called "LOCATION" with names of
                      each country, region, etc.
    :return: A pandas DataFrame with COVID-19 data for the
             input location.
    """
    df = pd.read_csv(data_file, sep='\t')
    try:
        df = df[df['LOCATION'] == location]
    except:
        print(f'Could not find data for location {location}...')
        return None
    return df

<!-- This allows us to re-use essentially the same code in many places simply by -->
<!-- calling the function with different input arguments. -->



# Exercise 2



**Part 1.**

As an example of how to visualize the data, we again use Afghanistan.
However, note that we will not spend time on interpreting the outbreak in
this country.

Before making any plots, we choose to extract the underlying NumPy arrays
holding the data on time and confirmed cases:

In [5]:
time = df['ELAPSED_TIME_SINCE_OUTBREAK'].to_numpy()
confirmed = df['CONFIRMED'].to_numpy()

Note that this is not really necessary, because we can also pass
`pandas.DataSeries` objects directly to the plotting commands

[Tip 1: If you are uncertain about what kind of an object a variable `x`
refers to, the command `type(x)` provides the answer! ]


**Part 2.**

To visualize the data for Afghanistan, we input the arrays from the
previous part to the `scatter` function provided by `matplotlib`:

In [6]:
fig, ax = plt.subplots()
ax.grid()
ax.set_xlabel('Time since initial outbreak (days)')
ax.set_ylabel('Number of confirmed cases')
ax.scatter(time, confirmed, color='black')

**Part 3.** and 4


Generalizing what we just did, we write a Python function to create plots for
an *arbitrary location*:

In [7]:
def plot_confirmed_cases(location, data_file='../data/corona_data.dat'):
    """
    Plots the number of confirmed COVID-19 cases for a specific
    location.

    :param location: The name of the location (case-sensitive).
    :param data_file: Path to file holding the COVID-19 data.
                      It is expected that columns of data are
                      separated by tabs, and that there is a
                      column called "LOCATION" with names of
                      each country, region, etc.
    :return: A matplotlib.pyplot.figure object.
    """
    # Get data
    df = get_corona_data(location, data_file)
    time = df['ELAPSED_TIME_SINCE_OUTBREAK'].to_numpy()
    confirmed = df['CONFIRMED'].to_numpy()

    # Make plot
    fig, ax = plt.subplots()
    ax.set_title(location)
    ax.grid()
    ax.set_xlabel('Time since initial outbreak (days)')
    ax.set_ylabel('Number of confirmed cases')
    ax.scatter(time, confirmed, color='black')
    return fig

Note that we return the figure object from the function; this allows us to
customize the plot further outside of the function should we wish to do so.


**Part 5.**

Next, we plot number of confirmed cases in Hubei and Norway:

In [8]:
fig_hubei = plot_confirmed_cases('Hubei')
fig_norway = plot_confirmed_cases('Norway')

To build intuition about the COVID-19 outbreak, we want to compare the data with
a simple mathematical model. Therefore, we will come back to these data
below, after having introduced our model of choice.

# Using the SI-model to describe the spread of COVID-19

Most models for epidemics are some sort of compartment model [[kermack1927contribution]](#kermack1927contribution).
These models first divide the total population into a set of boxes or compartments,
each of which represents a possible "disease state" (healthy, exposed, infected, recovered, vaccinated, dead, etc.)
Next, differential equations are set up to describe how individuals "flow"
from one compartment to another. Obviously, compartment models come in many
different flavours, and they have greatly varying complexity.
Since this is an introductory project, we will use an extremely simple
model, the *deterministic SI-model* [[batista2021simulation]](#batista2021simulation):

<!-- Equation labels as ordinary links -->
<div id="eq:SI_susceptible"></div>

$$
\begin{equation}
\label{eq:SI_susceptible} \tag{1}
\frac{\mathrm{d}S(t)}{\mathrm{d}t} = -\beta\cdot\frac{S(t)I(t)}{N} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SI_infected"></div>

$$
\begin{equation}  
\label{eq:SI_infected} \tag{2}
\frac{\mathrm{d}I(t)}{\mathrm{d}t} = +\beta\cdot\frac{S(t)I(t)}{N} \,.
\end{equation}
$$

In the above equations, $N$ is the total population size, $S(t)$ denotes the
number of susceptible people at time $t$, the population at risk of being
infected, while $I(t)$ denotes the number of infected (sick) individuals.
Since there are only two compartments, we must have $S(t)+I(t)=N$ at all times.
This implies that people never die or recover from the disease, and that
eventually everyone will be infected, a clear weakness with the model.


# Exercise 3
<div id="exercise:no_hope"></div>



**Part 1.**

The analytical solution to equation ([2](#eq:SI_infected)) can be derived by
first inserting $S(t)=N-I(t)$, separating variables, and integrating:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\displaystyle\int_{I_0}^{I(t)}\frac{\mathrm{d}I(t)}{I(t)(N-I(t))} \mathrm{d}I
= \displaystyle\int_{0}^{t}\frac{\beta}{N}\mathrm{d}t=\beta t\,,
\label{_auto1} \tag{3}
\end{equation}
$$

where $I_0=I(0)$. To calculate the integral on the left, we can use use partial
fractions to split the integrand into a sum of two parts:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\displaystyle\int_{I_0}^{I(t)}\left(\frac{1}{I} + \frac{1}{N-I}\right)
\mathrm{d}I = \left[\ln{I} - \ln{(N-I)}\right]_{I_0}^{I(t)} 
\label{_auto2} \tag{4}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}  
= \ln{(\frac{I}{N-I})}_{I_0}^{I(t)}  
\label{_auto3} \tag{5}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}  
= \ln{\frac{I(t)}{N-I(t)}} - \ln{\frac{I_0}{N-I_0}} 
\label{_auto4} \tag{6}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}  
= \ln{\frac{I(t)}{N-I(t)}} + \ln{\frac{S_0}{I_0}}\,.
\label{_auto5} \tag{7}
\end{equation}
$$

Hence, taking the exponential on both sides of the original integral:

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
\frac{I(t)}{N-I(t)} = \frac{I_0}{S_0}\cdot \exp{(\beta t)}  
\label{_auto6} \tag{8}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:It_analytical"></div>

$$
\begin{equation}  
\label{eq:It_analytical} \tag{9}
I(t) = \frac{S_0+I_0}{1+\frac{S_0}{I_0}\exp(-\beta{t})}\,.
\end{equation}
$$

where $S_0=S(0)$, and where we have used that $S_0 + I_0=N$.
Finally, the number of susceptible people are:

<!-- Equation labels as ordinary links -->
<div id="eq:St_analytical"></div>

$$
\begin{equation}
\label{eq:St_analytical} \tag{10}
S(t) = N - I(t) = \frac{(S_0+I_0)\frac{S_0}{I_0}\exp(-\beta{t})}{1+\frac{S_0}{I_0}\exp(-\beta{t})} \,.
\end{equation}
$$

# Exercise 4



**Part 1.**

The following Python function calculates the analytical solution
to the SI-model, equations ([10](#eq:St_analytical))
and ([9](#eq:It_analytical)):

In [9]:
def calc_SI_model(t, S0, I0, beta):
    """
    :param t: An array of times.
    :param S0: The initial number of susceptible people.
    :param I0: The initial number of infected people.
    :param beta: The disease transmission rate parameter.
    :return: A tuple of arrays holding S(t) and I(t).
    """
    I = (S0+I0)/(1.0 + S0*np.exp(-beta*t)/I0)
    S = S0 + I0 - I
    return S, I

**Part 2.**

Next, we implement a new function that combines the SI-model with the
plot function:

In [10]:
def compare_confirmed_cases_with_model(location, S0, I0, beta,
                                       data_file='../data/corona_data.dat'):
    """
    Plots the number of confirmed COVID-19 cases for a specific
    location.

    :param location: The name of the location (case-sensitive).
    :param S0: The initial number of susceptibles in the model.
    :param I0: The initial number of infected people in the model.
    :param beta: The model disease transmission rate parameter.
    :param data_file: Path to file holding the COVID-19 data.
                      It is expected that columns of data are
                      separated by tabs, and that there is a
                      column called "LOCATION" with names of
                      each country, region, etc.
    :return: A matplotlib.pyplot.figure object.
    """
    # Calculate S(t) and I(t) from the analytical solution
    t = np.linspace(0, 250, 251)
    St, It = calc_SI_model(t, S0, I0, beta)

    # Plot the data, and return the data
    fig = plot_confirmed_cases(location, data_file)
    # Add modelled I(t) to the same figure
    ax = fig.axes[0]
    ax.plot(t, It)
    return fig

**Part 3.**

The city of Wuhan is located in the province of Hubei. Since COVID-19 first
arose in Wuhan [[zhou2020pneumonia]](#zhou2020pneumonia), the data for Hubei is of special
interest when studying the dynamics of the disease transmission.
As remarked previously, the simple SI-model predicts that everyone
in the population will be infected eventually.
On the other hand, the data for Hubei indicates that the Corona
virus spread very quickly after the initial outbreak, but then reached a
plateau level much lower than the total population size.
Assuming the data to be trustworthy, this is most likely a consequence of
strong countermeasures being put in place in Hubei.

To match the model to the data, we therefore set $N$ equal to the final number
of confirmed cases in Hubei. For simplicity we also assume that at time $t=0$,
the number of infected individuals was $I_0=1$, hence $S_0=N-1$.
We manually adjust $S_0$ and $\beta$ to get a match between model and data:

In [11]:
df = get_corona_data('Hubei')
N = df['CONFIRMED'].iloc[-1]
fig_hubei_model = compare_confirmed_cases_with_model('Hubei', N-1, 1, 0.6)

**Part 4.**

For comparison purposes, it might be interesting to study how the virus
spread in a more confined space. One such example comes from the
cruise ship *Diamond Princess*.
There were $S_0=712$ confirmed cases of the Corona virus aboard
the cruise ship [[WikiDiamondPrincess]](#WikiDiamondPrincess).
If we use the same $\beta$-value we history-matched to the Hubei data,
we get:

In [12]:
compare_confirmed_cases_with_model('Diamond Princess', 712, 1, 0.6)

It should be remarked that the total number of passengers were
about five times larger [[WikiDiamondPrincess]](#WikiDiamondPrincess).


**Part 5.**

As with the case of Hubei, we cannot use the total population size of Norway
for $S_0$, because that would lead to millions of people being infected in
the model calculation.
A challenge with matching this particular data set is there is not a single
plateau level for the disease; instead, there is a second wave starting at
around 150 days. Therefore, the history-match will clearly be poorer for
Norway overall than for the previous two locations.

To make things simple, we ignore the second wave and only focus on the initial
period. After 150 days, the number of confirmed cases is close to 8000-9000,
so we choose $S_0=8000$:

In [13]:
compare_confirmed_cases_with_model('Norway', 8000, 1, 0.6)
compare_confirmed_cases_with_model('Norway', 8000, 1, 0.27)

As seen in the above two plots, we have to lower $\beta$ in order to
capture the trend in the data.
One interpretation is that, initially, the disease spread more slowly in
Norway than in Hubei. At the same time, the countermeasures were not as
effective in Norway.


# Exercise 5

We have seen that the $SI$-model works quite well provided that
the disease a) spreads very quickly, and b) levels out thereafter.
However, there are several clear weaknessnes with the model.
One big problem has to do with the fact that there is only two compartments
in the model; as remarked several times, this means that people
can never recover from the disease in the model, i.e., everyone will eventually
be infected. A more realistic model would add extra compartments, to account
for not only recovered individuals, but also dead people, vaccinated ones, etc.

Another issue has to do with the $\beta$-factor, which is a measure of the
"effective probability of infection". We have so far assumed $\beta$ to be
constant, but in reality we know that it should change with time.
While some factors could lead to a larger risk of disease transmission
(e.g., mutated variants of the virus [[adam2021scientists]](#adam2021scientists)),
it is likely that $\beta$ will decrease due to strong
countermeasures put in place by the world's different governments.
To capture this kind of behavior, we again use a simple model and
suppose that $\beta$ declines exponentially:

<!-- Equation labels as ordinary links -->
<div id="eq:p3:betaT"></div>

$$
\begin{equation}
\beta(t)=\beta_0 e^{-\lambda t}.
\label{eq:p3:betaT} \tag{11}
\end{equation}
$$

One can prove that equations similar to ([9](#eq:It_analytical))
and ([10](#eq:St_analytical)) still hold; all we have to do is to
replace the product $\beta t$ with

<!-- Equation labels as ordinary links -->
<div id="eq:l"></div>

$$
\begin{equation}
\beta t \to \int_0^t\beta_0e^{-\lambda t}dt=\frac{\beta_0}{\lambda}(1-e^{-\lambda t}).
\label{eq:l} \tag{12}
\end{equation}
$$

In the above expression, $\beta_0$ is the initial infection rate,
while the exponenial decline parameter, $\lambda$, is used to capture
effects of counter-measures: A high value of $\lambda$ indicates strong
disease-prevention, while in the limit $\lambda\rightarrow{0}$ we recover
our original SI-model that has no intervention effects.



**Part 1.**

We implement the exponential decline model by modifying the
functions we made previously: we include both $\beta$ and $\lambda$ as
input arguments, and simply let $\lambda=0$ by default:

[Tip 2: If you choose this way of implementing your code,
you are allowed to skip straight to the "most advanced version"
in your handed-in project.
This way you avoid copying and pasting almost identical code in your project,
and you reduce the risk of introducing bugs!]

In [14]:
# We replace the old function "calc_SI_model" by this:
def calc_SI_model(t, S0, I0, beta, lam=0.0):
    """
    :param t: An array of times.
    :param S0: The initial number of susceptible people.
    :param I0: The initial number of infected people.
    :param beta: The disease transmission rate parameter.
    :param lam: Decline parameter for exponential decline
                of beta (default: 0.0).
    :return: A tuple of arrays holding S(t) and I(t).
    """
    bt = (beta/lam)*(1.0 - np.exp(-lam*t)) if lam > 0 else beta*t
    I = (S0+I0)/(1.0 + S0*np.exp(-bt)/I0)
    S = S0 + I0 - I
    return S, I

Similarly, we update our last plotting function in two places by passing
in the extra input argument:

In [15]:
# Replace the old "comparison function" with this:
def compare_confirmed_cases_with_model(location,
                                       S0,
                                       I0,
                                       beta,
                                       lam=0.0,
                                       data_file='../data/corona_data.dat'):
    """
    Plots the number of confirmed COVID-19 cases for a specific
    location.

    :param location: The name of the location (case-sensitive).
    :param S0: The initial number of susceptibles in the model.
    :param I0: The initial number of infected people in the model.
    :param beta: The model disease transmission rate parameter.
    :param lam: Decline parameter for exponential decline
                of beta (default: 0.0).
    :param data_file: Path to file holding the COVID-19 data.
                      It is expected that columns of data are
                      separated by tabs, and that there is a
                      column called "LOCATION" with names of
                      each country, region, etc.
    :return: A matplotlib.pyplot.figure object.
    """
    # Calculate S(t) and I(t) from the analytical solution
    t = np.linspace(0, 250, 251)
    St, It = calc_SI_model(t, S0, I0, beta, lam)

    # Plot the data, and return the data
    fig = plot_confirmed_cases(location, data_file)
    # Add modelled I(t) to the same figure
    ax = fig.axes[0]
    ax.plot(t, It)
    return fig

**Part 2.**

The actual population size of Hubei has been reported
as 57.8 million [[ChinaPopulation]](#ChinaPopulation). Let us try using this value
as input to the exponential-decline SI-model. By repeated trial-and-error,
we find a combination that works reasonably well:

In [16]:
compare_confirmed_cases_with_model('Hubei', 57.8e6, 1, 2.0, 0.18)

Note that we have more than tripled the initial $\beta$-factor compared to
our previous match (now $\beta_0=2$ vs. before 0.6) We are still able to
capture the trend in the data, though, by tuning $\lambda$ accordingly.
This example illustrates the fact that different models may fit the same
data equally well, and that values of fitted parameters do not always
have a clear-cut "physical interpretation". However, as long as we apply the
*same model to different locations* we may compare the fitted parameters and
hopefully learn something useful.

The updated SI-model behaves differently from the original one in that the
disease outbreak will not reach the whole population.
However, it is still the case that the model can only account a single wave.
This is because $\beta(t)\rightarrow{0}$ as $t\rightarrow{\infty}$,
which means that after some time, no one will be infected anymore,
regardless of how many infectious individuals there are in the population.


**Part 3.**

The population of Norway is approximately 5.4 million [[NorwayPopSSB]](#NorwayPopSSB).
If we use the $\beta-$ and $\lambda-$values fitted to the Hubei data in order
to predict the disease outbreak in Norway, we get:

In [17]:
compare_confirmed_cases_with_model('Hubei', 5.4e6, 1, 2.0, 0.18)

We see that we drastically overpredict the number of COVID-19 cases in Norway.


**Part 4.**

By trial-and-error, we find an improved match for the initial disease
transmission period in Norway:

In [18]:
compare_confirmed_cases_with_model('Norway', 5.4e6, 1, 0.8, 0.0898)

If we can assume that the value of $\lambda$ represents the government
response, this means that while the initial outbreak was more severe in
Hubei than in Norway, countermeasures were less strict in Norway
($\beta_{0, \text{Hubei}}$ > $\beta_{0, \text{Norway}}$, $\lambda_\text{Norway}$ < $\lambda_\text{Hubei}$), which fits
well with what has been reported in the media [[CovidGovResponseTracker]](#CovidGovResponseTracker).

# Conclusion and Discussion


It is surprising that the SI-model can describe the data so well.
Countries are very different, but still the spread of COVID-19
are captured using similar model parameters for different places.
When interpreting the data, we  can therefore compare different countries
quantitatively, e.g., how fast the disease spreads, and the strength of
governmental responses.
We find that the governmental countermeasures were much stronger in Hubei than
in Norway. However, many more countries should be analyzed with the model
before we can conclude that an exponential decline model for $\beta$ makes
sense.

The work in this report supports the notion that we can use compartment
models to analyze the Corona virus outbreak. To make the model more realistic,
additional compartments should be introduced, to account for people are
infected without being sick (asymptomatics), people who have recovered from
the virus, dead people, vaccinated people, and so on.
For large countries, or for very spread-out populations, compartments could also
be introduced for different regions, and then one should allow for some sort of
non-neighbour flow between them to account for traffic (buses, trains, airplanes etc.)

# Self-reflections


**Aksel:** I started too late on the projects, next time I will start immediately, visit the lab every Wednesday and work continuously. We worked quite well as a team. I enjoyed to see how we could use a model to analyze data, and extract knowledge. However, I think that the model is too simple to capture the complexity of how an infectious disease spreads. It would be very interesting to see how the model could be improved to capture more effects, like vaccination, social distancing, etc.

**Oddbjørn:** I worked on the project in-between several other tasks, and I did most of the writing work close to the deadline. Though I think the project went reasonably well, it is probably better if I start earlier next time. Being a former mathematics student, I really enjoyed learning about how the Corona virus can be modelled with differential equations. The project allowed also gave me appreciation for how difficult it can be to analyze real-world data; it would be exciting to continue in the future by gradually extending the complexity of the model to allow for more realism!


# Bibliography
1. <div id="SarsCov2"></div> **C. S. G. o. t. I. C. o. T. o. Viruses**. 
    The Species Severe Acute Respiratory Syndrome-Related Coronavirus: Classifying 2019-NCoV and Naming It SARS-CoV-2,
    *Nature microbiology*,
    5(4),
    pp. 536,
    2020.

2. <div id="zhou2020pneumonia"></div> **P. Zhou, X.-L. Yang, X.-G. Wang, B. Hu, L. Zhang, W. Zhang, H.-R. Si, Y. Zhu, B. Li, C.-L. Huang et al.**. 
    A Pneumonia Outbreak Associated With a New Coronavirus of Probable Bat Origin,
    *Nature*,
    579(7798),
    pp. 270-273,
    2020.

3. <div id="kermack1927contribution"></div> **W. O. Kermack and A. G. McKendrick**. 
    A Contribution to the Mathematical Theory of Epidemics—I,
    *Proceedings of the Royal Society of London. Series A, Containing papers of a mathematical and physical character*,
    115(772),
    pp. 700-721,
    1927.

4. <div id="batista2021simulation"></div> **A. M. Batista, S. L. Souza, K. C. Iarosz, A. C. Almeida, J. D. Szezech, E. C. Gabrick, M. Mugnaine, G. L. d. Santos and I. L. Caldas**. 
    Simulation of Deterministic Compartmental Models for Infectious Diseases Dynamics,
    *arXiv preprint arXiv:2106.02085*,
    2021.

5. <div id="WikiDiamondPrincess"></div> **Wikipedia**. 
    COVID-19 Pandemic on Diamond Princess,
    2021,
    Accessed: 2021-08-17,
    <https://en.wikipedia.org/wiki/COVID-19_pandemic_on_Diamond_Princess>.

6. <div id="adam2021scientists"></div> **D. Adam**. 
    What Scientists Know About New, Fast-Spreading Coronavirus Variants.,
    *Nature*,
    2021.

7. <div id="ChinaPopulation"></div> **N. B. o. S. o. China**. 
    Communiqué of the Seventh National Population Census[1] (No. 3) - Population by Region,
    November 1, 2020,
    Accessed: 2021-08-17,
    <http://www.stats.gov.cn/english/PressRelease/202105/t20210510_1817188.html>.

8. <div id="NorwayPopSSB"></div> **S. Norway**. 
    Population in Norway, 2nd Quarter 2021,
    Updated: 19 August 2021,
    Accessed: 2021-08-19,
    <https://www.ssb.no/en/befolkning/folketall/statistikk/befolkning>.

9. <div id="CovidGovResponseTracker"></div> **U. o. O. B. S. o. Government**. 
    COVID-19 GOVERNMENT RESPONSE TRACKER,
    2021,
    Accessed: 2021-08-17,
    <https://www.bsg.ox.ac.uk/research/research-projects/covid-19-government-response-tracker>.